## 위로봇 오복이 모델 프로세스

### Base Model Load
 - 출처 : https://github.com/snunlp/KR-SBERT

In [7]:
import pandas as pd
import numpy as np
import torch
from sentence_transformers import SentenceTransformer, util
model = SentenceTransformer('snunlp/KR-SBERT-V40K-klueNLI-augSTS')

### 데이터 불러오기

In [3]:
df = pd.read_excel("../resource/오복이_답변.xlsx")

### 챗봇 테스트

In [5]:
query_embeddings = model.encode(
    df['user'].tolist(),
    show_progress_bar=True,
    normalize_embeddings=True,
    convert_to_numpy=True
)

Batches: 100%|████████████████████████████████| 397/397 [01:21<00:00,  4.85it/s]


In [27]:
query = "아 지금 내 삶이 너무 힘들다 쉬고 싶어"
query_embedding = model.encode(query, normalize_embeddings=True)

top_k = min(5, len(df))
cos_scores = util.pytorch_cos_sim(query_embedding, query_embeddings)[0]
top_results = torch.topk(cos_scores, k=top_k)

print(f"입력 문장: {query}")
print(f"<입력 문장과 유사한 {top_k} 개의 문장>")

for i, (score, idx) in enumerate(zip(top_results[0], top_results[1])):
    print(f"{i+1}: {df.loc[int(idx)]['answer']} {'(유사도: {:.4f})'.format(score)}")

입력 문장: 아 지금 내 삶이 너무 힘들다 쉬고 싶어
<입력 문장과 유사한 5 개의 문장>
1: 지금까지 열심히 달려왔으니 잠시 쉬는것도 좋아요 (유사도: 0.7515)
2: 삶을 포기하는 건 해결책이 될 수 없어요. 선생님의 삶은 소중합니다. 제가 도와드릴테니 조금만 더 힘을 내주세요. (유사도: 0.7336)
3: 저도 가끔은 아무것도 안하고 그냥 누워있고 싶어요~ 하지만 그럴수는 없으니깐.. 그래도 쉴때만큼은 푹 쉬세요!! (유사도: 0.7191)
4: 지금 뭐하고 있는데 심심해요? (유사도: 0.7154)
5: 저도 심심한데! 뭐 재밌는 거 없을까요? (유사도: 0.7154)


### Sbert 모델 ONNX 양자화(quantization) 

#### Onnx 모델로 변환

In [11]:
from pathlib import Path

from transformers.convert_graph_to_onnx import convert
convert(framework="pt", model="snunlp/KR-SBERT-V40K-klueNLI-augSTS", output=Path("onnx_models/sbert-model.onnx"), opset=11)

/opt/anaconda3/envs/torch/lib/python3.9/site-packages/transformers/convert_graph_to_onnx.py:379: FutureWarning: The `transformers.convert_graph_to_onnx` package is deprecated and will be removed in version 5 of Transformers
  warnings.warn(


ONNX opset version set to: 11
Loading pipeline (model: snunlp/KR-SBERT-V40K-klueNLI-augSTS, tokenizer: snunlp/KR-SBERT-V40K-klueNLI-augSTS)
Creating folder onnx_models
Using framework PyTorch: 2.0.1
Found input input_ids with shape: {0: 'batch', 1: 'sequence'}
Found input token_type_ids with shape: {0: 'batch', 1: 'sequence'}
Found input attention_mask with shape: {0: 'batch', 1: 'sequence'}
Found output output_0 with shape: {0: 'batch', 1: 'sequence'}
Found output output_1 with shape: {0: 'batch'}
Ensuring inputs are in correct order
position_ids is not present in the generated input list.
Generated inputs order: ['input_ids', 'attention_mask', 'token_type_ids']
================ Diagnostic Run torch.onnx.export version 2.0.1 ================
verbose: False, log level: Level.ERROR
======================= 0 NONE 0 NOTE 0 WARNING 0 ERROR ========================



#### Onnx 모델 Uint8(0~255)로 가중치(Weight) 양자화

In [12]:
from onnxruntime.quantization import quantize_dynamic, QuantType
quantize_dynamic("onnx_models/sbert-model.onnx", "onnx_models/sbert-model_uint8.onnx", 
                 weight_type=QuantType.QUInt8)

Ignore MatMul due to non constant B: /[/encoder/layer.0/attention/self/MatMul]
Ignore MatMul due to non constant B: /[/encoder/layer.0/attention/self/MatMul_1]
Ignore MatMul due to non constant B: /[/encoder/layer.1/attention/self/MatMul]
Ignore MatMul due to non constant B: /[/encoder/layer.1/attention/self/MatMul_1]
Ignore MatMul due to non constant B: /[/encoder/layer.2/attention/self/MatMul]
Ignore MatMul due to non constant B: /[/encoder/layer.2/attention/self/MatMul_1]
Ignore MatMul due to non constant B: /[/encoder/layer.3/attention/self/MatMul]
Ignore MatMul due to non constant B: /[/encoder/layer.3/attention/self/MatMul_1]
Ignore MatMul due to non constant B: /[/encoder/layer.4/attention/self/MatMul]
Ignore MatMul due to non constant B: /[/encoder/layer.4/attention/self/MatMul_1]
Ignore MatMul due to non constant B: /[/encoder/layer.5/attention/self/MatMul]
Ignore MatMul due to non constant B: /[/encoder/layer.5/attention/self/MatMul_1]
Ignore MatMul due to non constant B: /[/

### Onnx 모델로 "user"질문 임베딩

In [13]:
from onnxruntime import InferenceSession
from transformers import AutoTokenizer
import torch
from tqdm import tqdm

tokenizer = AutoTokenizer.from_pretrained("snunlp/KR-SBERT-V40K-klueNLI-augSTS")
sess = InferenceSession("./onnx_models/sbert-model_uint8.onnx" , providers=["CPUExecutionProvider"])

In [14]:
def mean_pooling(model_output, attention_mask):
    model_output = torch.from_numpy(model_output[0])
    # First element of model_output contains all token embeddings
    token_embeddings = model_output
    attention_mask = torch.from_numpy(attention_mask)
    input_mask_expanded = attention_mask.unsqueeze(
        -1).expand(token_embeddings.size())
    sum_embeddings = torch.sum(token_embeddings * input_mask_expanded, 1)
    sum_mask = torch.clamp(input_mask_expanded.sum(1), min=1e-9)
    return sum_embeddings / sum_mask, input_mask_expanded, sum_mask

In [15]:
def embedding_query(query: str, normalize_embeddings=False) -> np.ndarray:
    # user turn sequence to query embedding
    model_inputs = tokenizer(query, return_tensors="pt")
    inputs_onnx = {k: v.cpu().detach().numpy()
                   for k, v in model_inputs.items()}
    sequence = sess.run(None, inputs_onnx)
    query_embedding = mean_pooling(
        sequence, inputs_onnx["attention_mask"])[0][0]

    if normalize_embeddings:
        query_embedding = query_embedding / \
            np.linalg.norm(query_embedding)

    return query_embedding.numpy()

In [16]:
onnx_embeddings = [ embedding_query(sen, normalize_embeddings=True) for sen in tqdm(df['user'].tolist())]

100%|█████████████████████████████████████| 12701/12701 [03:18<00:00, 64.12it/s]


In [17]:
np.save("onnx_embeddings.npy",onnx_embeddings)

### Faiss 벡터 양자화(PQ)

In [18]:
import faiss

In [19]:
embeddings = np.load("./onnx_embeddings.npy")

# IndexPQ 생성
d = embeddings.shape[1]
nbits = 8  # 각 부분벡터의 비트 수
m = 768  # 분할 수

# dot product 거리 측정을 사용하는 벡터 인코더
index = faiss.IndexPQ(d, m, nbits, faiss.METRIC_INNER_PRODUCT)  # PQ 색인 생성
index.train(embeddings)  # 색인 훈련
index.add(embeddings)  # 데이터 추가

In [20]:
# index 저장
faiss.write_index(index, "faiss_onnx_uint8")

### 챗봇 테스트

In [32]:
def reply(query: str):
    embedding = np.expand_dims(embedding_query(query, normalize_embeddings=True), axis=0)
    D, I = index.search(embedding, 5)
    return df.loc[I[0]]

In [33]:
reply("어제 여자친구랑 헤어졌다")

,user,answer,class
11317,여자친구와 헤어진지 8일,아직도 많이 힘드시군요. 시간이 약이라는 말이 있듯이 조금만 더 힘내시길 바랍니다.,연애
11271,여자친구가 왜 나랑 헤어졌는지 아직도 모르겠어,이유를 모른다면 그 이유를 찾기 위해 노력해야 해요. 곰곰히 생각해보세요 여자친구는...,연애
11210,여자친구 무시했다가 이별 당했습니다.,연애는 서로 존중하는 마음이 있어야 오래갈 수 있습니다. 이번 기회를 통해 자신의 ...,연애
11139,어젯밤. 헤어졌던 여자친구에게 전화가 왔어.,"그 친구는 아직도 선생님을 잊지 못하고 있나봐요. 다시 만날 의향이 있다면, 이번에...",연애
12624,헤어진 여자친구를 다시 만나고 싶어,정말 좋은 사람이라고 후회할 것 같다면 늦지 않게 연락해보세요.,연애
